<a href="https://colab.research.google.com/github/j-ranasinghe/nlp_cw/blob/vishmi/Solution_GroupH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CM4603 – Coursework 1 (Group)
#### November 2023

### Group: H

Name 1: Anuttara Rajasinghe |
IIT ID: 20210216 |
RGU ID: 2117946

Name 2: < insert your name > |
IIT ID: < insert your IIT ID > |
RGU ID: < insert your RGU ID >

Name 3: Vishmi Herath |
IIT ID: 20210503 |
RGU ID: 2117528

### Answer to Task 1

    Describe the dataset, any insights you gained about the data and the effectiveness of the different (subword) tokenization schemes tried out.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

# Replace 'path_to_your_file' with the actual path in your Google Drive
file_path = '/content/drive/MyDrive/NLP CW/full-combined-dataset.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)


In [7]:
# Extract only the specified columns
filtered_data = []
for entry in data:
    filtered_entry = {
        "title": entry.get("title"),
        "lang": entry.get("lang"),
        "rating": entry.get("rating"),
        "helpfulVotes": entry.get("helpfulVotes"),
        "text": entry.get("text"),
        "PlaceName": entry.get("placeInfo", {}).get("name"),
        "PlaceRating": entry.get("placeInfo", {}).get("rating")
    }
    filtered_data.append(filtered_entry)


In [8]:
filtered_data

[{'title': 'Terrible experience. Waste of time, money and nerves. Stay away.',
  'lang': 'en',
  'rating': 1,
  'helpfulVotes': 1,
  'text': 'Well, no.   Big no. \nFirst of all, before booking, please go through all these comments. Look at the pictures. Not posted by the property but from guest. The place have a great potential but as of today it is just a scam. Place is undeveloped, rooms are nasty. Food terrible. All three above aspects can be fixed easily and reading that this "in a jungle" makes no sense. We travel to Sri Lanka often, staying in boutique hotels but also in a real jungle. All places can be clean and nice disputing the fact of not easy environment around. Same w/ food. Staying "in a jungle" explain nasty food served? Having so many option for low-key or upscale Sri Lanka\' culinary possibilities? \n\nSo we did claim many times issues on site. Zero interest. Management\'s reaction: it is what it is, deal with it. \nImmediate decision to leave. After one night in dirty

In [9]:
print(len(filtered_data))

10127


Explain the challenges encountered and the observations based on the output above

### Answer to Task 2

Describe the classification dataset, its dimensions, preprocessing steps, numbers of classes taken for classification, the which feature extraction techniques were performed.

In [ ]:
# Insert code for Task 2

Describe the final dataset shapes of the featuresets extracted.

### Answer to Task 3

Justify the 3 non-deep learning algorithms you are using for classification.

In [ ]:
# Insert code for Task 3

Compare and interpret the results of the different algorithms and feature extraction methods on the text classification task.

### Answer to Task 4

Describe the contextual embedding used and it's effect on the performance of the algorithms selected in Task 3.

In [ ]:
# Insert code for Task 4

Compare the deep learning algorithm results with that of the 3 non-deep learning algorithms and interpret the overal results